<a href="https://colab.research.google.com/github/AKHIRANANDHINI/NLP/blob/main/Lab%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, MaxPooling1D, Flatten, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
import zipfile # Import the zipfile module

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

# Corrected line to read 'train.csv' from the zip archive
with zipfile.ZipFile('/content/archive (7) (1).zip', 'r') as z:
    with z.open('train.csv') as f:
        df = pd.read_csv(f)
print(df.head())

# Corrected column names based on df.head() output
texts = df["description_x"].values
labels = df["same_security"].astype(int).values # Convert boolean labels to integers

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS]
    return " ".join(tokens)

# Apply clean_text to the correct column
df["clean_text"] = df["description_x"].apply(clean_text)

X_train, X_val, y_train, y_val = train_test_split(df["clean_text"], labels, test_size=0.2, random_state=42)

count_vec = CountVectorizer()
X_train_count = count_vec.fit_transform(X_train)
X_val_count = count_vec.transform(X_val)

tfidf_vec = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vec.fit_transform(X_train)
X_val_tfidf = tfidf_vec.transform(X_val)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_val_tfidf)

svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)
y_pred_svm = svm.predict(X_val_tfidf)

print("\n--- Logistic Regression (TF-IDF) ---")
print(classification_report(y_val, y_pred_lr))

print("\n--- SVM (TF-IDF) ---")
print(classification_report(y_val, y_pred_svm))

tokenizer = Tokenizer(num_words=10000, oov_token="")
tokenizer.fit_on_texts(df["clean_text"])
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

max_len = 30
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post")
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding="post")

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

mlp = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])
mlp.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
mlp.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_mlp = (mlp.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- MLP (Embeddings) ---")
print(classification_report(y_val, y_pred_mlp))

cnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    Conv1D(128, 5, activation="relu"),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])
cnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
cnn.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_cnn = (cnn.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- CNN (Embeddings) ---")
print(classification_report(y_val, y_pred_cnn))

lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(128, dropout=0.3, recurrent_dropout=0.3),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])
lstm.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
lstm.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_lstm = (lstm.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- LSTM (Embeddings) ---")
print(classification_report(y_val, y_pred_lstm))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   Unnamed: 0                                      description_x  \
0           0                     first trust dow jones internet   
1           1                schwab intl large company index etf   
2           2                       vanguard small cap index adm   
3           3  duke energy corp new com new isin #us4 sedol #...   
4           4                                   visa inc class a   

                                       description_y ticker_x ticker_y  \
0                        first trust dj internet idx      FDN      FDN   
1  schwab strategic tr fundamental intl large co ...     FNDF     FNDF   
2                 vanguard small-cap index fund inst    VSMAX    VSCIX   
3  duke energy corp new com new isin #us26441c204...      DUK      DUK   
4                                          visa inc.        V        V   

   same_security  
0           True  
1           True  
2          False  
3           True  
4           True  

--- Logistic Regression (TF-IDF

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.7156 - loss: 0.5949 - val_accuracy: 0.7436 - val_loss: 0.5522
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7502 - loss: 0.5470 - val_accuracy: 0.7436 - val_loss: 0.5325
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7497 - loss: 0.5243 - val_accuracy: 0.7436 - val_loss: 0.4832
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7632 - loss: 0.4540 - val_accuracy: 0.8135 - val_loss: 0.3878
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8572 - loss: 0.3553 - val_accuracy: 0.8834 - val_loss: 0.3186
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

--- MLP (Embeddings) ---
              precision    recall  f1-score   support

           0       0.82      0.70      0.75       110
           1       0.90      0.95      0.92       319

    accuracy                           0.88       429
   macro avg       0.86      0.82      0.84       429
weighted avg       0.88      0.88   

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.7197 - loss: 0.5587 - val_accuracy: 0.8695 - val_loss: 0.3449
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8783 - loss: 0.3230 - val_accuracy: 0.8951 - val_loss: 0.3160
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9202 - loss: 0.2162 - val_accuracy: 0.8928 - val_loss: 0.3270
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9378 - loss: 0.1761 - val_accuracy: 0.8858 - val_loss: 0.3309
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9558 - loss: 0.1236 - val_accuracy: 0.8788 - val_loss: 0.3466
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

--- CNN (Embeddings) ---
              precision    recall  f1-score   support

           0       0.76      0.76      0.76       110
           1       0.92      0.92      0.92       319

    accuracy                           0.88       429
   macro avg       0.84      0.84      0.84       429
weighted avg       0.88      0.88      0.8

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


54/54 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.7299 - loss: 0.6007 - val_accuracy: 0.7436 - val_loss: 0.5725
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.7628 - loss: 0.5527 - val_accuracy: 0.7436 - val_loss: 0.4902
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.8282 - loss: 0.4070 - val_accuracy: 0.8415 - val_loss: 0.3864
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.8701 - loss: 0.3076 - val_accuracy: 0.8415 - val_loss: 0.3788
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - accuracy: 0.8727 - loss: 0.3100 - val_accuracy: 0.8765 - val_loss: 0.3527
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step

--- LSTM (Embeddings) ---
              precision    recall  f1-score   support

           0       0.75      0.78      0.76       110
           1       0.92      0.91      0.92       319

    accuracy                           0.88       429
   macro avg       0.84      0.85      0.84       429
weighted avg       0.88      0.88      